In [1]:
import sys

from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sys.path.append("/home/aseliverstov/projects/brain_signals/experiments/src")
from src.dataset import EEG

2025-01-27 23:14:12.360992: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 23:14:12.377810: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 23:14:12.391012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738008852.408342   15544 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738008852.417401   15544 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 23:14:12.445599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/dataset")
intencities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
n_channels = 31

dataset = EEG(data_dir=data_dir, intencities=intencities, n_channels=n_channels)
dataset.load_to_dataframe(users=[1, 2, 3, 4, 5], signals=["delta", "theta", "alpha", "beta", "gamma"])

X, y, ids = dataset.X, dataset.y, dataset.ids

# Replication

In [7]:
train_ids = (ids["signal_type"] != "delta").to_numpy()
test_ids = (ids["signal_type"] == "delta").to_numpy()

X_train, y_train = X[train_ids], y[train_ids]
X_test, y_test = X[test_ids], y[test_ids]

In [8]:
import numpy as np
from scipy.stats import kurtosis, skew

def calculate_summary_features(arr, n_windows=30):
    features = []

    cur_arr = np.array_split(arr, n_windows, axis=1)
    for window in cur_arr:
        features.append(np.mean(window, axis=1))
        features.append(np.mean(np.abs(window), axis=1))
        features.append(np.std(window, axis=1))
        features.append(np.sqrt(np.mean(np.square(window), axis=1)))
        features.append(kurtosis(window, axis=1, fisher=True))
        features.append(skew(window, axis=1))

    features = np.column_stack(features)    
    return features

In [9]:
X_train = calculate_summary_features(X_train)
X_test = calculate_summary_features(X_test)

X_train.shape

(6200, 180)

In [10]:
model = Sequential()
model.add(Dense(64, input_shape= (X_train.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=10, batch_size=10, verbose=1)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │        11,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,690 (61.29 KB)

 Trainable params: 15,690 (61.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1187 - loss: 2.4051 - val_accuracy: 0.1852 - val_loss: 2.9554
Epoch 2/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2132 - loss: 2.1777 - val_accuracy: 0.2135 - val_loss: 3.6516
Epoch 3/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2649 - loss: 2.0691 - val_accuracy: 0.2310 - val_loss: 4.2491
Epoch 4/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3111 - loss: 1.9525 - val_accuracy: 0.2594 - val_loss: 4.0842
Epoch 5/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3324 - loss: 1.9062 - val_accuracy: 0.2400 - val_loss: 5.4296
Epoch 6/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3572 - loss: 1.8482 - val_accuracy: 0.2484 - val_loss: 7.1064
Epoch 7/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3705 - loss: 1.8081 - val_accuracy: 0.2665 - val_loss: 6.0127
Epoch 8/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4056 - loss: 1.7415 - val_accuracy: 0.